In [1]:
# from mtcnn import MTCNN
import cv2
import matplotlib.pyplot as plt

import os
import requests
import torch

from torch import autocast
from torch.nn import functional as F
from torchvision import transforms
from diffusers import (
    StableDiffusionPipeline, AutoencoderKL,
    UNet2DConditionModel, PNDMScheduler, LMSDiscreteScheduler
)
from diffusers.schedulers.scheduling_ddim import DDIMScheduler
from transformers import CLIPTextModel, CLIPTokenizer, CLIPProcessor, CLIPModel
from tqdm.auto import tqdm
from huggingface_hub import notebook_login
from huggingface_hub import HfApi, HfFolder
from PIL import Image, ImageDraw
from diffusers import StableDiffusionPipeline
import torch
import numpy as np
import pickle
from scipy import signal

import numpy as np
import matplotlib.pyplot as plt

import os 
from tqdm import tqdm 

device = 'cuda'
PATH = 'dataset//'

2023-02-22 09:41:44.620689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def preprocess(pil_image):
    pil_image = pil_image.convert("RGB")
    processing_pipe = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ])
    tensor = processing_pipe(pil_image)
    tensor = tensor.reshape(1, 3, 512, 512)
    return tensor


def encode_vae(img):
    img_tensor = preprocess(img)
    with torch.no_grad():
        diag_gaussian_distrib_obj = vae.encode(img_tensor.to(device), return_dict=False)
        img_latent = diag_gaussian_distrib_obj[0].sample().detach().cpu()
        img_latent *= 0.18215
    return img_latent

#Загрузка сети, создание функции генерации по вектору 
vae = AutoencoderKL.from_pretrained(
    'runwayml/stable-diffusion-v1-5', 
    subfolder='vae',
    use_auth_token=True
)
vae = vae.to(device)
# dict(vae.config)

def decode_latents(latents):
    latents = 1 / 0.18215 * latents

    with torch.no_grad():
        images = vae.decode(latents)['sample']

    images = (images / 2 + 0.5).clamp(0, 1)
    images = images.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (images * 255).round().astype('uint8')
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images


#Функции интерполяции

def interpolate(tensor1, tensor2, curve, num_points):
    points = []
    for i in range(num_points):
        alpha = i / (num_points - 1)
        weight = curve(alpha)
        seem_chaos = 0.2
        tensor = ((1 - weight) * tensor1) + weight * tensor2
        #1!!!!!
#         tensor+=torch.randn(1,4,64,64)*seem_chaos
        points.append(tensor)
    return points

def sine_curve(alpha):
    return 0.5 * (np.sin(np.pi * (alpha - 0.5)) + 1)



#Функция для создания синуса с возможностью контролировать в простратсве 
def shifted_sin(x, x_offset=0.94, y_offset=0,freq=-4):
    return np.sin(freq * x - x_offset) + y_offset

#Функция для создания синуса с шумом и сглажииванием 
def sine_curve_with_smooth_noise(alpha, noise_scale=0, filter_alpha=0.8):
    noise = np.random.normal(scale=noise_scale, size=(1))
    smoothed_noise = signal.lfilter([1 - filter_alpha], [1, -filter_alpha], noise)
    return (0.5 * (np.sin(np.pi * (alpha - 0.5)) + 1) + smoothed_noise)[0]

#Функция для модулирования сигнала 
def modulation_two_sin(x):
    return sine_curve_with_smooth_noise(x)*shifted_sin(x)
    



In [ ]:
# lattens = []
# for file in tqdm(files):
#     img = Image.open(f"dataset/{file}")
#     img = img.resize((512,512))
#     lattens.append(encode_vae(img))
    

In [ ]:
#Строим график функции интерполяции  если нужно 
# alphas= np.linspace(0, 1 , 100)

# y = [modulation_two_sin(alpha) for alpha in alphas]

# plt.plot(alphas, y)
# plt.xlabel('x')
# plt.ylabel('y')
# plt.title('y_max= ' + str(y[-1]))
# plt.show()

In [ ]:
# тест картинки после множественного денойзинга 
# img = decode_latents(lattens[0].to(device))
# display(img[0])
# for i in range(10):
#     vector1=  encode_vae(img[0])
#     img = decode_latents(vector1.to(device))
#     print(i)
#     display(img[0])

In [ ]:
#Тест интерполяции 2 векторов по нашей кривой 
# tensor1 = vector1
# tensor2 = vector2
# points = interpolate(tensor1, tensor2, sine_curve_with_smooth_noise, 20)
# for i in points:
#     img = decode_latents(i.to(device))
#     display(img[0])

## Начало конвеера 


In [ ]:
#Первое я бы расчитал все вектора которые нужны и сохранил бы в picke 
#Пердполагаемый формат данных {"img_name":[векторы от 0 до 20 ]}
#После находим стартовый вектор у которого больше всего соседий начинаем с него pipeline 
#Берем все его вектора после находим ближайший вектор к последнему для него 
#Берем все вектора у соседа задонаперед с срезом первых пяти 
#Далее находим у 5 соседа 

In [4]:
# Шумоподавление картинки и подача самой себя 20 раз
#Сохранение векторов на кажом шаге 
NUM_ITERATION_DENOISING=20
files = os.listdir(PATH)
main_lattens = {}

for file in tqdm(files):
    lattens=[]
    img = Image.open(f"{PATH}{file}")
    img = img.resize((512,512))
    #первый вектор 
    lattens.append(encode_vae(img))
    img = decode_latents(lattens[0].to(device))
    for i in tqdm(range(NUM_ITERATION_DENOISING)):
        vector =  encode_vae(img[0])
        img = decode_latents(vector.to(device))
        lattens.append(vector)
    main_lattens [file] = lattens

with open('oleg.pickle', 'wb') as f:
    pickle.dump(main_lattens, f)
    

  0%|                                                                        | 0/68 [00:08<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   12 │   lattens.append(encode_vae(img))                                                         │
│   13 │   img = decode_latents(lattens[0].to(device))                                             │
│   14 │   for i in tqdm(range(NUM_ITERATION_DENOISING)):                                          │
│ ❱ 15 │   │   vector =  encode_vae(img[0])                                                        │
│   16 │   │   img = decode_latents(vector.to(device))                                             │
│   17 │   │   lattens.append(vector)                                                              │
│   18 │   main_lattens [file] = lattens                                                           │
│                                                                                                  │
│ in encode_vae                                                                                    │
│                                                                                                  │
│   14 │   img_tensor = preprocess(img)                                                            │
│   15 │   with torch.no_grad():                                                                   │
│   16 │   │   diag_gaussian_distrib_obj = vae.encode(img_tensor.to(device), return_dict=False)    │
│ ❱ 17 │   │   img_latent = diag_gaussian_distrib_obj[0].sample().detach().cpu()                   │
│   18 │   │   img_latent *= 0.18215                                                               │
│   19 │   return img_latent                                                                       │
│   20                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [5]:
#Загружаем заранее посчитанные вектора и с ними и работаем 
with open('main_lattens.pickle', 'rb') as f:
    main_lattens = pickle.load(f)

In [6]:
def find_start_vector(data):
    """
    Находит вектор, ближайший к центру группы векторов.
    С него начнем 
    Аргументы:
    data -- numpy массив данных размерности (n_samples, 1, 4, 64, 64).
    Возвращает:
    closest_vector -- numpy массив размерности (1, 4, 64, 64), представляющий
    вектор, ближайший к центру группы векторов.
    """
    # Вычисляем центр группы векторов
    center = np.mean(data, axis=0)
    # Приводим данные и центр к формату (n_samples, n_features), где n_features = 1*4*64*64 = 16384
    data = data.reshape(data.shape[0], -1)
    center = center.reshape(1, -1)
    # Вычисляем расстояние между центром и каждым вектором
    distances = np.linalg.norm(data - center, axis=1)
    # Находим индекс вектора с минимальным расстоянием до центра
    min_distance_idx = np.argmin(distances)
    # Получаем вектор с минимальным расстоянием до центра
    closest_vector = data[min_distance_idx, :]
    # Приводим вектор обратно к исходному формату (1, 4, 64, 64)
    closest_vector = closest_vector.reshape(1, 4, 64, 64)
    return closest_vector, min_distance_idx

In [7]:
chose_start_vectors = []
for i in main_lattens.keys():
#     chose_start_vectors[i]=main_lattens[i][0][0].numpy()
    chose_start_vectors.append(main_lattens[i][0][0].numpy()) 
#     chose_start_vectors.append(main_lattens[i][0])
x = np.array(chose_start_vectors)
# find_closest_vector(x)


In [8]:
import numpy as np

def find_nearest_neighbors(query_vector, pool_vectors, k=5):
    """
    Находит k ближайших соседей к заданному вектору в пуле векторов.
    
    Аргументы:
    query_vector -- numpy массив размерности (1, 4, 64, 64), представляющий
    вектор запроса.
    pool_vectors -- numpy массив размерности (n_samples, 1, 4, 64, 64), представляющий
    пул векторов.
    k -- количество ближайших соседей, которые нужно найти. По умолчанию k=5.
    
    Возвращает:
    indices -- numpy массив размерности (k,), содержащий индексы k ближайших
    соседей в пуле векторов.
    distances -- numpy массив размерности (k,), содержащий расстояния от
    запроса до каждого из k ближайших соседей.
    """
    
    # Приводим вектор запроса и пул векторов к формату (n_samples, n_features),
    # где n_features = 1*4*64*64 = 16384
    query_vector = query_vector.reshape(1, -1)
    pool_vectors = pool_vectors.reshape(pool_vectors.shape[0], -1)

    # Вычисляем расстояние между запросом и каждым вектором в пуле
    distances = np.linalg.norm(pool_vectors - query_vector, axis=1)

    # Находим индексы k векторов с наименьшим расстоянием до запроса
    indices = np.argsort(distances)[:k]

    # Получаем расстояния до k ближайших соседей
    distances = distances[indices]

    return indices, distances


In [10]:
#Новый подход 
# У нас есть 2 массива 5iter с ключем изображения  и 20iter с ключем изображения 
#Сперва находим ближайший к стартовому в 5iter, берем все кадры в animation_frame 
#Последний используем для нахождения ближайшего в 20iter разворачиваем его  и кладем в animation_frame 
#После добовляем интерполяцию 
DENOISING_TRASHHOLD=5
twenty_iter = {}
five_iter = {}
for key in main_lattens:
    five_iter[key] = main_lattens[key][DENOISING_TRASHHOLD].numpy()
    twenty_iter[key] = main_lattens[key][-1].numpy()




In [11]:
#Находим ключ ближайшего фото для start_vector
def key_nearest_vector_dict(vec,target_dict):
    index,_ = find_nearest_neighbors(vec,np.array(list(target_dict.values())))
    return list(target_dict.keys())[index[0]]
    

In [12]:
x = np.array(chose_start_vectors)
start_vector , _ = find_start_vector(x)

### Основной блок анимации 

In [15]:
DENOISING_TRASHHOLD=5
twenty_iter = {}
five_iter = {}
for key in main_lattens:
    five_iter[key] = main_lattens[key][DENOISING_TRASHHOLD].numpy()
    twenty_iter[key] = main_lattens[key][-1].numpy()


animation_list = []
new_start_vec = start_vector
for i in tqdm(range(len(five_iter)-1)):
    
    key = key_nearest_vector_dict(new_start_vec,five_iter)
    

   
    #Берем все следующии кадры 
    new_frames = main_lattens[key][DENOISING_TRASHHOLD:]
    
    #Место для интерполяции 
    # start_vector вместе  new_frames[0]
    points = interpolate(new_start_vec, new_frames[0].numpy(),
                         sine_curve,16)
    for i in points[::-1]:
        new_frames.insert(0,i)
    
    #Добовляем в 
    
    
    
    #Удаляем из всех словарей старый ключ 
    del five_iter[key]
    del twenty_iter[key]

    #нАзначаем новый вектор 
    new_start_vec = new_frames[-1].numpy()
    #добовляем new_frames в список кадров анимации 
    [animation_list.append(i) for i in new_frames]
    
    if len(five_iter) == 0:
        break
    
    key = key_nearest_vector_dict(new_start_vec,twenty_iter)

    #Добавляем все кадры с новым ключем срезая на 5 и переворачивая последовательность 
    new_frames = list(reversed(main_lattens[key][DENOISING_TRASHHOLD:]))
    
    points = interpolate(new_start_vec, new_frames[0].numpy(),
                         sine_curve, 16)
    for i in points[::-1]:
        new_frames.insert(0,i)
    
    #
    del five_iter[key]
    del twenty_iter[key]
    
    #нАзначаем новый вектор 
    new_start_vec = new_frames[-1].numpy()

    #добовляем new_frames в список кадров анимации 
    [animation_list.append(i) for i in new_frames]
    if len(five_iter) == 0:
        break



 49%|██████████████████████████████▌                               | 33/67 [00:00<00:00, 333.55it/s]


In [17]:
!rm -rf output/
!mkdir output
show_animation_list(animation_list)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   1 get_ipython().system('rm -rf output/')                                                       │
│   2 get_ipython().system('mkdir output')                                                         │
│ ❱ 3 show_animation_list(animation_list)                                                          │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'show_animation_list' is not defined

In [41]:
!ffmpeg -y -framerate 12 -i /home/amigutskiy/Workbench/art_poster/output/frame_%d.jpg  -c:v libx264 -pix_fmt yuv420p -crf 18 final_SD.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [43]:
#Поставил синсус
from IPython.display import Video
Video(filename="final_SD.mp4")

In [201]:
#Поставил синсус
Video(filename="final_30.mp4")

In [199]:
#Поставил синсус
Video(filename="final_15.mp4")

In [192]:
# Нужно сделать возможность сохранения 
def show_animation_list(animation_list, save=True):
    for count, frame in enumerate(tqdm(animation_list)):
        vec = torch.tensor(frame)
        img = decode_latents(vec.to(device))[0]
        if save:
            img.save(f"output/frame_{str(count)}.jpg",quality=100)
        if not save:
            display(img)

        

In [63]:
show_animation_list("wer")

  0%|                                                                                                                 | 0/3 [00:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 show_animation_list("wer")                                                                   │
│   2                                                                                              │
│                                                                                                  │
│ in show_animation_list                                                                           │
│                                                                                                  │
│    1 # Нужно сделать возможность сохранения                                                      │
│    2 def show_animation_list(animation_list, save=True):                                         │
│    3 │   for count, frame in enumerate(tqdm(animation_list)):                                    │
│ ❱  4 │   │   vec = torch.tensor(frame)                                                           │
│    5 │   │   img = decode_latents(vec.to(device))[0]                                             │
│    6 │   if save:                                                                                │
│    7 │   │   print("ок")                                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: new(): invalid data type 'str'

In [ ]:
show_animation_list(animation_list)

In [ ]:
#Фильтруем данные для упрощения беру у всех [5:] 
#Если захочишь добавить в начле больше оригинала сделай это руками
DENOISING_TRASHHOLD=4
big_vectors_bag = []
for values in tqdm(main_lattens.values()):
    #Срезаю 5 первых генераций как не интересные
    for frame in values[DENOISING_TRASHHOLD:]:
        big_vectors_bag.append(frame.numpy())
#     new_lst = [elem for sublist in lst for elem in sublist]
    
big_vectors_bag = np.array(big_vectors_bag)

In [ ]:
#Попытка расчета анимации
#Результат большой корзины мне не нрав, нужно искать ближайший только среди чистых и шумных
target_vector = start_vector
animation_list = [] 
test_dist=[]
# for i in range(len(big_vectors_bag)):
for i in range(100):
    #Ищем вектор, добовляем его в лист, удаляем его ищим следующий 
    indexes, dist = find_nearest_neighbors(target_vector,  big_vectors_bag,k=5)
    test_dist.append(dist[0])
    target_vector = big_vectors_bag[indexes[0]]
    animation_list.append(target_vector)
    big_vectors_bag = np.delete(big_vectors_bag,indexes[0],axis=0)
    print(indexes)
    print(dist)
    show_animation_list([target_vector])
    
    